In [82]:
import os
from onai.ml.tools.spark import get_spark
import numpy as np

import nbimporter
from Data_API_PS_Demo import Pandas_to_Dict

%store -r Six_digit_NAICS
%store -r Data_Set

In [19]:
spark = get_spark()

In [20]:
def spark_load(spark,p,extra_options = None,persist = False,temp_view_name= None):
    
    fmt, remained_p = p.split("://", 1)
    reader = spark.read
    if extra_options:
        reader = reader.options(**extra_options)
    if fmt in SUPPORTED_FMT_LIST:
        reader = reader.format(fmt)
        p = remained_p
    res_df = reader.load(p)
    if persist:
        res_df = res_df.persist()
    if temp_view_name is not None:
        res_df.createOrReplaceTempView(temp_view_name)
        
    return res_df

SUPPORTED_FMT_LIST = {'orc', 'parquet', 'csv'}

In [21]:
business_dir = 's3://one-lake-prod/ml-snapshots/20220116/business'

spark_load(spark, os.path.join(business_dir, 'financial_data'), temp_view_name='fd')
spark_load(spark, os.path.join(business_dir, 'company_data_denormalized'), temp_view_name='cdd')
spark_load(spark, os.path.join(business_dir, 'exchange_rate'), temp_view_name='er')
spark_load(spark, os.path.join(business_dir, 'company_data'), temp_view_name='cd')

spark_load(spark, 'csv://' + os.path.join(business_dir, 'dime_map', 'dime.csv'), 
           dict(header='True', inferSchema='True', delimiter=','), temp_view_name='dm')

DataFrame[oneid: string, mnemonic: string, provider_name: string, provider_data_item_id: string, name: string, accounting_concept_code: string, is_symbol: boolean, is_monetary: boolean, is_fiscal: boolean, is_array: boolean, value_type: string, description: string]

In [41]:
query = """select cdd.entity_id,cdd.primary_naics_node,dm.mnemonic,event_date, fd.data_value from fd 
        join dm on dm.oneid = fd.data_item_id
        join cdd on cdd.entity_id = fd.entity_id
        where dm.mnemonic in ('TOTAL_REVENUE','EBITDA','NET_DEBT_EBITDA','TOTAL_DEBT_EQUITY')
        and fd.period_type = 'ANNUAL'
        and dm.provider_name = 'CapitalIQ'
        and cdd.country_of_incorporation_iso = 'USA'
        order by entity_id, fd.event_date desc
        """

df = spark.sql(query)
CapIQ_data = df.toPandas()

CapIQ_data = Pandas_to_Dict(CapIQ_data)

In [2]:
for i in CapIQ_data.items():
    print(i)
    break

NameError: name 'CapIQ_data' is not defined

In [118]:
def Eliminate_Empty_data(CapIQ_data):    
    l = []
    for i in CapIQ_data.items():
        ID,data = i
        if len(data['Revenue']) < 3 or len(data['EBITDA']) < 3 or len(data['NDE']) < 3 or len(data['TDE']) < 3:
            l.append(ID)


    for i in l:
        CapIQ_data.pop(i)
        
    return CapIQ_data

CapIQ_data_dict = Eliminate_Empty_data(CapIQ_data)

print(len(CapIQ_data_dict),'\n')
n = 0
for i in CapIQ_data_dict.items():
    print(i)
    break

12774 

('00017f58-baec-51fd-9a90-e2acae968457', {'sector': '722511', 'Revenue': [('1996', 128044405), ('1995', 120259849), ('1994', 102796965)], 'EBITDA': [('1997', 15472548), ('1996', 15514546), ('1995', 14214730), ('1994', 16634335)], 'NDE': [('1997', 0.503775), ('1996', 0.918293), ('1995', 1.277082)], 'TDE': [('1997', 18.9853), ('1996', 27.4103), ('1995', 30.4835)], 'R_Time': ['1996', '1995', '1994'], 'E_Time': ['1997', '1996', '1995', '1994'], 'N_Time': ['1997', '1996', '1995'], 'T_Time': ['1997', '1996', '1995']})


In [121]:
def Get_latest_Five_Year_Data(CapIQ_data):
    
    FYear_Data_Dict = {}
    for i in CapIQ_data.items():
        ID,data = i
        FYear_Data_Dict[ID] = {}
        FYear_Data_Dict[ID]['Sector'] = data['sector']
        FYear_Data_Dict[ID]['Revenue'] = data['Revenue'][:20][::-1]
        FYear_Data_Dict[ID]['EBITDA'] = data['EBITDA'][:20][::-1]
        FYear_Data_Dict[ID]['NDE'] = data['NDE'][:20][::-1]
        FYear_Data_Dict[ID]['TDE'] = data['TDE'][:20][::-1]
            
    l = []
    Time_Dict = {}
    for i in FYear_Data_Dict.items():
        ID,data = i

        R_time = []
        for j in data['Revenue']:
            R_time.append(j[0])

        E_time = []
        for j in data['EBITDA']:
            E_time.append(j[0])

        N_time = []
        for j in data['NDE']:
            N_time.append(j[0])

        T_time = []
        for j in data['TDE']:
            T_time.append(j[0])

        lis = list(set(R_time) & set(E_time) & set(N_time) & set(T_time))
        lis.sort()
        Time_Dict[ID] = lis
        
        
    l = []
    for i in Time_Dict.items():
        ID,time = i
        if len(time) < 5:
            l.append(ID)

    for i in l:
        Time_Dict.pop(i)
            
        
        
    Five_Year_Data_Dict = {}
    for i in Time_Dict.items():
        ID,time = i
        data = FYear_Data_Dict[ID]
        Five_Year_Data_Dict[ID] = {}
        Five_Year_Data_Dict[ID]['Sector'] = data['Sector']

        Five_Year_Data_Dict[ID]['R'] = []
        for j in time[:5]:
            for p in data['Revenue']:
                if j == p[0]:
                    Five_Year_Data_Dict[ID]['R'].append(p[1])

        Five_Year_Data_Dict[ID]['EBITDA'] = []
        for j in time[:5]:
            for p in data['EBITDA']:
                if j == p[0]:
                    Five_Year_Data_Dict[ID]['EBITDA'].append(p[1])

        Five_Year_Data_Dict[ID]['NDE'] = []
        for j in time[:5]:
            for p in data['NDE']:
                if j == p[0]:
                    Five_Year_Data_Dict[ID]['NDE'].append(p[1])

        Five_Year_Data_Dict[ID]['TDE'] = []
        for j in time[:5]:
            for p in data['TDE']:
                if j == p[0]:
                    Five_Year_Data_Dict[ID]['TDE'].append(p[1])
                        
                        
    return Five_Year_Data_Dict


CapIQ_data_dict_FY = Get_latest_Five_Year_Data(CapIQ_data)

print(len(CapIQ_data_dict_FY),'\n')
for i in CapIQ_data_dict_FY.items():
    print(i)
    break

8814 

('00023c5a-004b-53a9-b57f-fca73cca257f', {'Sector': '721110', 'R': [6342229, 9376101, 12966184, 18229748, 34668532], 'EBITDA': [5206512, 8236829, 11413999, 16114194, 30351273], 'NDE': [5.757914, 2.684077, 2.565823, 3.301254, 5.648098], 'TDE': [138.8867, 43.9635, 39.4153, 53.9335, 126.0586]})


In [122]:
def Data_Set_Normalised(Data_Set):
    
    Data_Set_N = {}
    
    for i in Data_Set.items():
        ID,data = i
        
        Data_Set_N[ID] = {'Sector': data['Sector']}
        
        Data_Set_N[ID].update({'Revenue' : []})
        for j in data['R']:
            if j < 0:
                Data_Set_N[ID]['Revenue'].append(0)
            elif j in range(0,5000000):
                Data_Set_N[ID]['Revenue'].append(round((0.1 + (j)/50000000),3))
            elif j in range(5000000,10000000):
                Data_Set_N[ID]['Revenue'].append(round((0.2 + (j-5000000)/50000000),3))
            elif j in range(10000000,50000000):
                Data_Set_N[ID]['Revenue'].append(round((0.3 + (j)/500000000),3))
            elif j in range(50000000,100000000):
                Data_Set_N[ID]['Revenue'].append(round((0.4 + (j-50000000)/500000000),3))
            elif j in range(100000000,500000000):
                Data_Set_N[ID]['Revenue'].append(round((0.5 + (j)/5000000000),3))
            elif j in range(500000000,1000000000):
                Data_Set_N[ID]['Revenue'].append(round((0.6 + (j-500000000)/5000000000),3))
            elif j in range(1000000000,5000000000):
                Data_Set_N[ID]['Revenue'].append(round((0.7 + (j)/50000000000),3))
            elif j in range(5000000000,10000000000):
                Data_Set_N[ID]['Revenue'].append(round((0.8 + (j-5000000000)/50000000000),3))
            elif j > 10000000000:
                Data_Set_N[ID]['Revenue'].append(round((0.9 + (j)/500000000000),3))
                
        
        Data_Set_N[ID].update({'EBITDA' : []})
        for j in data['EBITDA']:
            if j < 0:
                Data_Set_N[ID]['EBITDA'].append(0)
            elif j in range(0,5000000):
                Data_Set_N[ID]['EBITDA'].append(0.1)
            elif j in range(5000000,10000000):
                Data_Set_N[ID]['EBITDA'].append(0.2)
            elif j in range(10000000,50000000):
                Data_Set_N[ID]['EBITDA'].append(0.3)
            elif j in range(50000000,100000000):
                Data_Set_N[ID]['EBITDA'].append(0.4)
            elif j in range(100000000,500000000):
                Data_Set_N[ID]['EBITDA'].append(0.5)
            elif j in range(500000000,1000000000):
                Data_Set_N[ID]['EBITDA'].append(0.6)
            elif j in range(1000000000,5000000000):
                Data_Set_N[ID]['EBITDA'].append(0.7)
            elif j in range(5000000000,10000000000):
                Data_Set_N[ID]['EBITDA'].append(0.8)
            elif j > 10000000000:
                Data_Set_N[ID]['EBITDA'].append(0.9)
                
            
        Data_Set_N[ID].update({'NDE' : []})
        for j in data['NDE']:
            if j < 0 or j > 1:
                Data_Set_N[ID]['NDE'].append(0)
            else:
                Data_Set_N[ID]['NDE'].append(round(1 - j,4))
          
        
        Data_Set_N[ID].update({'TDE' : []})
        for j in data['TDE']:
            if j < 0 or j > 100:
                Data_Set_N[ID]['TDE'].append(0)
            else:
                Data_Set_N[ID]['TDE'].append(round((100 - j)/100,4))
                
        
        
    return Data_Set_N

In [124]:
CapIQ_data_dict_FY_N = Data_Set_Normalised(CapIQ_data_dict_FY)

for i in CapIQ_data_dict_FY_N.items():
    ID ,data = i
    print(i)
    break
    
    
%store CapIQ_data_dict_FY_N

('00023c5a-004b-53a9-b57f-fca73cca257f', {'Sector': '721110', 'Revenue': [0.227, 0.288, 0.326, 0.336, 0.369], 'EBITDA': [0.2, 0.2, 0.3, 0.3, 0.3], 'NDE': [0, 0, 0, 0, 0], 'TDE': [0, 0.5604, 0.6058, 0.4607, 0]})
Stored 'CapIQ_data_dict_FY_N' (dict)


In [72]:
def NAICS_count(CapIQ_data):
    NAICS_count = {}
    for i in Six_digit_NAICS:
        for j in CapIQ_data.items():
            ID,data = j
            
            if data['Sector'] == i:
                if data['Sector'] in NAICS_count.keys():
                    NAICS_count[data['Sector']] += 1
                else:
                    NAICS_count[data['Sector']] = 1

    return NAICS_count

FY_N_count = NAICS_count(CapIQ_data_dict_FY)

In [109]:
L = []
n = 0
for i in FY_N_count.items():
    s,c = i
    if c > 20:
        print(s,c)
        n += c
        L.append(s)
print(len(L),n) 

%store L

213111 23
213112 63
221112 21
221118 21
221122 32
221210 77
221310 23
237210 22
324110 49
325411 21
325412 270
325413 23
325414 193
325620 32
326199 23
333132 34
333242 22
333249 25
334112 23
334118 43
334210 32
334220 43
334290 49
334310 25
334413 152
334419 52
334510 76
334511 33
334519 55
335999 27
336390 32
336413 29
339112 131
339113 45
339920 31
423430 26
423690 21
445110 24
454110 43
486210 57
511210 476
512110 37
515120 35
515210 47
517311 73
517312 52
517911 41
517919 26
518210 125
519130 108
524113 66
524114 38
524126 94
525990 21
531110 40
531120 40
532490 23
541330 23
541511 34
541512 73
541715 38
541810 32
621511 21
621610 24
621999 22
622110 24
713210 29
721110 57
721120 35
722511 68
722513 42
71 3887
Stored 'L' (list)


In [75]:
print(len(FY_N_count))

712


In [55]:
def Get_latest_Three_Year_Data(CapIQ_data):
    
    FYear_Data_Dict = {}
    for i in CapIQ_data.items():
        ID,data = i
        FYear_Data_Dict[ID] = {}
        FYear_Data_Dict[ID]['Sector'] = data['sector']
        FYear_Data_Dict[ID]['Revenue'] = data['Revenue'][:20][::-1]
        FYear_Data_Dict[ID]['EBITDA'] = data['EBITDA'][:20][::-1]
        FYear_Data_Dict[ID]['NDE'] = data['NDE'][:20][::-1]
        FYear_Data_Dict[ID]['TDE'] = data['TDE'][:20][::-1]
            
    l = []
    Time_Dict = {}
    for i in FYear_Data_Dict.items():
        ID,data = i

        R_time = []
        for j in data['Revenue']:
            R_time.append(j[0])

        E_time = []
        for j in data['EBITDA']:
            E_time.append(j[0])

        N_time = []
        for j in data['NDE']:
            N_time.append(j[0])

        T_time = []
        for j in data['TDE']:
            T_time.append(j[0])

        lis = list(set(R_time) & set(E_time) & set(N_time) & set(T_time))
        lis.sort()
        Time_Dict[ID] = lis
        
    l = []
    for i in Time_Dict.items():
        ID,time = i
        if len(time) < 3:
            l.append(ID)

    for i in l:
        Time_Dict.pop(i)
            
        
        
    Five_Year_Data_Dict = {}
    for i in Time_Dict.items():
        ID,time = i
        data = FYear_Data_Dict[ID]
        Five_Year_Data_Dict[ID] = {}
        Five_Year_Data_Dict[ID]['Sector'] = data['Sector']

        Five_Year_Data_Dict[ID]['R'] = []
        for j in time[:3]:
            for p in data['Revenue']:
                if j == p[0]:
                    Five_Year_Data_Dict[ID]['R'].append(p[1])

        Five_Year_Data_Dict[ID]['EBITDA'] = []
        for j in time[:3]:
            for p in data['EBITDA']:
                if j == p[0]:
                    Five_Year_Data_Dict[ID]['EBITDA'].append(p[1])

        Five_Year_Data_Dict[ID]['NDE'] = []
        for j in time[:3]:
            for p in data['NDE']:
                if j == p[0]:
                    Five_Year_Data_Dict[ID]['NDE'].append(p[1])

        Five_Year_Data_Dict[ID]['TDE'] = []
        for j in time[:3]:
            for p in data['TDE']:
                if j == p[0]:
                    Five_Year_Data_Dict[ID]['TDE'].append(p[1])
                        
                        
    return Five_Year_Data_Dict


CapIQ_data_dict_TY = Get_latest_Three_Year_Data(CapIQ_data)

print(len(CapIQ_data_dict_TY),'\n')
for i in CapIQ_data_dict_TY.items():
    print(i)
    break

11875 

('00023c5a-004b-53a9-b57f-fca73cca257f', {'Sector': '721110', 'R': [6342229, 9376101, 12966184], 'EBITDA': [5206512, 8236829, 11413999], 'NDE': [5.757914, 2.684077, 2.565823], 'TDE': [138.8867, 43.9635, 39.4153]})


In [57]:
TY_N_count = NAICS_count(CapIQ_data_dict_TY)

L = []
for i in TY_N_count.items():
    s,c = i
    if c > 20:
        L.append(s)
print(len(L)) 

99
